In [69]:
%store -r columns_encoded
%store -r columns
%store -r data
%store -r data_encoded

columns_encoded = columns_encoded
columns = columns
data = data
data_encoded = data_encoded

In [70]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

pd.set_option('display.max_columns', None)

In [71]:
#1-5 Fragen mit Regression und Klassifikation
#
#

#GradientBoostingClassifier XGBoost

In [72]:
# Remove Pseudonym
data_encoded = data_encoded.drop(columns_encoded.pseudonym, axis=1)
columns_encoded = columns_encoded.drop('pseudonym')
data_encoded.replace()
# Change Geschlecht to nummeric entries
new_column_name = 'Geschlecht (weiblich?)'
data_encoded[columns_encoded.geschlecht] = [sex == 'weiblich' for sex in data_encoded.Geschlecht]
data_encoded.rename(columns={columns_encoded.geschlecht: new_column_name})
columns_encoded.geschlecht = new_column_name

# One Hot Encode Funktion und Studiengang Spalte
prefix_funktion = "funktion_"
prefix_studiengang = "studiengang_"

new_columns_funktion = data_encoded[columns_encoded.funktion].str.get_dummies(sep=";")
new_columns_funktion.columns = new_columns_funktion.columns.map(lambda x: prefix_funktion + x)
new_columns_studiengang = data_encoded[columns_encoded.studiengang].str.get_dummies(sep=";")
new_columns_studiengang.columns = new_columns_studiengang.columns.map(lambda x: prefix_funktion + x)

new_columns = new_columns_funktion + new_columns_studiengang

data_encoded = data_encoded.join(new_columns)
data_encoded = data_encoded.drop(columns_encoded.funktion, axis=1)
data_encoded = data_encoded.drop(columns_encoded.studiengang, axis=1)

In [73]:
columns_encoded

alter                                                                                                  Alter
geschlecht                                                                            Geschlecht (weiblich?)
funktion                                                   Welcher Funktion an der HSD fühlst du dich am ...
studiengang                                                Welchem Studiengang fühlst du dich am meisten ...
sportlichkeit                                                 Als wie sportlich würdest du dich einschätzen?
                                                                                 ...                        
lieblingsessen_Dampfnudeln                                                            lieblingsmusiker_Sting
lieblingsessen_Lachs mit Bratkartoffeln und Gurkensalat                    lieblingsmusiker_System of a Down
lieblingsmusiker_6lack                                                             lieblingsmusiker_The 1975
lieblingsmusiker_Ba

In [74]:
data_encoded.head()

,Alter,Geschlecht,Welchem Studiengang fühlst du dich am meisten zugehörig?,Als wie sportlich würdest du dich einschätzen?,Welche Rolle spielt für dich gesunde Ernährung?,Wie gerne gehst du Essen oder Trinken?,Wie gerne kochst du selbst?,Wie gerne gehst du feiern?,Wie sehr interessierst du dich für Musik?,Wie gerne liest Du?,"Wie sehr interessierst du dich für darstellende Künste\nwie Schauspiel, Tanz oder Gesang?","Wie sehr interessierst du sich für Filme, Kino, Fernsehsendungen?","Wie sehr interessierst du dich für bildende Kunst wie\nMalen, Zeichnen oder Fotografie?",Wie gerne besuchst du Museen?,Wie sehr interessierst du dich für Videospiele?,Wie häufig nutzt du soziale Medien?,"Wie groß ist dein Interesse an Technologie und Innovation,\nwie beispielsweise Programmierung, Robotik und Virtual Reality?","Wie groß ist dein Interesse an Astronomie, Weltraumforschung\noder astronomiebezogenen Aktivitäten?","Wie groß ist dein Interesse an Naturwissenschaften\nwie Biologie, Chemie und Physik?",Wie sehr interessierst du sich für gesellschaftliches\nEngagement und Freiwilligenarbeit?,"Wie groß ist dein Interesse an sozialen Themen wie\nVielfalt, Gleichberechtigung und soziale Gerechtigkeit?",Würdest du dich selbst als introvertiert/extrovertiert\nbezeichnen?,"Hast du häufig Angst etwas zu verpassen, wenn du nicht\nunter Menschen bist?",Wie … schätzt Du dich selbst ein? (1 gar nicht - 5\nsehr) [spontan],Wie … schätzt Du dich selbst ein? (1 gar nicht - 5\nsehr) [verwirrt],Wie … schätzt Du dich selbst ein? (1 gar nicht - 5\nsehr) [glücklich],Wie … schätzt Du dich selbst ein? (1 gar nicht - 5\nsehr) [geduldig],Wie … schätzt Du dich selbst ein? (1 gar nicht - 5\nsehr) [streitlustig],Wie … schätzt Du dich selbst ein? (1 gar nicht - 5\nsehr) [abenteuerlustig],"Hattest du schon einmal Zweifel daran, ob du dein Studium\nschaffen kannst?",Wie zufrieden bist du mit deinem Studium?,Strebst du nach dem Studium auch weiterhin eine akademische\nLaufbahn an?,Wie gern lernst Du in der Gruppe?,"Prokrastinierst du, wenn du lernen müsstest?",Wie häufig machst du Abgaben auf den letzten Drücker?,Wie gerne probierst du neue Lebensmittel aus?,Wie häufig verbringst du Zeit in der Natur (z. B. beim\nWandern oder Camping)?,Wie häufig verreist du?,Wie interessiert bist du an handwerklichen Tätigkeiten?,Wie gerne magst du Sprachnachrichten?,Wie wichtig ist es dir viel Geld zu verdienen?,sportarten_Basketball,sportarten_Bouldern/Klettern,sportarten_Football,sportarten_Fußball,sportarten_Handball,sportarten_Hockey,sportarten_Kampfsport,sportarten_Leichtathletik,sportarten_Reiten,sportarten_Rennrad/Mountainbike,sportarten_Schwimmen,sportarten_Tanzen,sportarten_Tennis,sportarten_Volleyball,sportarten_Wassersport,sportarten_Wintersport,sportarten_Yoga,lieblingssportart_Badminton,lieblingssportart_Basketball,lieblingssportart_Bogenschießen,lieblingssportart_Bouldern,lieblingssportart_Calisthenics,lieblingssportart_E-Sport,lieblingssportart_Fitness,lieblingssportart_Football,lieblingssportart_Fußball,lieblingssportart_Hockey,lieblingssportart_Inlineskaten,lieblingssportart_Joggen,lieblingssportart_Kampfsport,lieblingssportart_Karate,lieblingssportart_Keine Lieblingssportart,lieblingssportart_Klettern,lieblingssportart_Kraftsport,lieblingssportart_Padel-Tennis,lieblingssportart_Radfahren,lieblingssportart_Reiten,lieblingssportart_Ruderboot fahren,lieblingssportart_Schnorcheln,lieblingssportart_Schwimmen,lieblingssportart_Ski fahren,lieblingssportart_Squash,lieblingssportart_Tanzen,lieblingssportart_Tauchen,lieblingssportart_Tennis,lieblingssportart_Thaiboxen,lieblingssportart_Tischtennis,lieblingssportart_Trampolin,lieblingssportart_Volleyball,lieblingssportart_Yoga,lieblingsessen_Asiatische Nudeln,lieblingsessen_Burger,lieblingsessen_Burrito,lieblingsessen_Butterbrot,lieblingsessen_Carbonara,lieblingsessen_Cordon bleu,lieblingsessen_Curry,lieblingsessen_Dampfnudeln,lieblingsessen_Döner,lieblingsessen_Gemüsepfanne mit Reis,lieblingsessen_Gurke mit Mozzarella und Tofu,

# Frage 1: Wie häufig nutzt du soziale Medien? (Regression & Klassifikation)
Im folgenden werden wir verschiedene Machine Learning Modelle testen, um die Frage "" mithilfe der anderen Fragen zu schätzen. Da diese Frage sowohl als Regressions- als auch Klassifikationsproblem angesehen werden kann, nutzen wir Modelle für beide Varianten und vergleichen die Ergbnisse im Weiteren.

In [20]:
#Split label vom Rest der Daten
y_data_q1 = data_encoded[columns_encoded.social_media]
X_data_q1 = data_encoded.drop(columns_encoded.social_media, axis=1)

In [21]:
from sklearn.model_selection import train_test_split

# Train-test split
X_train_q1, X_test_q1, y_train_q1, y_test_q1 = train_test_split(X_data_q1, y_data_q1, test_size=0.2, random_state=0)

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error, make_scorer

## Suche nach dem besten Regressionsmodell

### Linear Regression

In [31]:
from sklearn.linear_model import LinearRegression

# Pipeline definieren
pipe_LR = Pipeline([
    ("scale", StandardScaler()),
    ("model", LinearRegression())
])

In [32]:
pipe_LR.fit(X_train_q1, y_train_q1)

ValueError: could not convert string to float: 'männlich'

### k-NN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Pipeline definieren
pipe_kNNR = Pipeline([
    ("scale", StandardScaler()),
    ("model", KNeighborsRegressor())
])

In [ ]:
# Grid Search nach den besten Parametern für das Modell k-NN Regressor
grid = GridSearchCV(estimator=pipe_kNNR,
                   param_grid={
                     'model__n_neighbors': [1, 2, 4, 5, 10, 20, 25],
                     'model__algorithm': ['ball_tree', 'kd_tree', 'brute', 'auto'],
                     'model__leaf_size': [3, 10, 20, 30, 50, 100],
                     'model__p': [1, 2, 3],
                     'model__metric': ['minkowski', 'precomputed']
                   },
                   scoring={"MAE": make_scorer(mean_absolute_error,
                                              greater_is_better=False)},
                   cv=3, refit="MAE")

grid.fit(X_data_q1, y_data_q1)

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipe_RFR = Pipeline([
    ("scale", StandardScaler()),
    ("model", RandomForestRegressor())
])

In [ ]:
grid_RFR = GridSearchCV(estimator=pipe_RFR,
                   param_grid={
                     'model__n_estimators': [20, 50, 100, 200],
                     'model__max_depth': [2, 3, 4, 6, 8, 10, 20],
                   },
                   scoring={"MAE": make_scorer(mean_absolute_error,
                                              greater_is_better=False)},
                   cv=3, refit="MAE", verbose=2)

grid_RFR.fit(X_train_q1, y_train_q1)

### Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

pipe_GBR = Pipeline([
    ("scale", StandardScaler()),
    ("model", GradientBoostingRegressor())
])

### Ergebnis:

## Suche nach dem besten Klassifikationsmodell

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipe_RFC = Pipeline([
    ("scale", StandardScaler()),
    ("model", RandomForestClassifier())
])

### k-NN Classifier

In [29]:
from sklearn.neighbors import KNeighborsClassifier

pipe_kNNC = Pipeline([
    ("scale", StandardScaler()),
    ("model", KNeighborsClassifier())
])

### Gradient Boosting Classifier

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

pipe_GBC = Pipeline([
    ("scale", StandardScaler()),
    ("model", GradientBoostingClassifier())
])

### Ergebnis:


## Bestes Modell für Frage 1

# Frage 2: Welche Sportarten interessieren dich am meisten (wähle maximal 5)? Sportart Fußball (Klassifikation)
Im folgenden werden wir verschiedene Machine Learning Modelle testen, um die Frage "Welche Sportarten interessieren dich am meisten (wähle maximal 5)?" bezüglich der Sportart Fußball mithilfe der anderen Fragen zu schätzen. Dafür werden verschiedene Klassifikationsmodelle von uns verglichen, um das beste Modell zu ermitteln.

# Frage 3: Alter? (Regression)
Im folgenden werden wir verschiedene Machine Learning Modelle testen, um das Alter mithilfe der anderen Fragen zu schätzen. Dafür werden verschiedene Regressionsmodelle von uns verglichen, um das beste Modell zu ermitteln.